In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import pandas as pd
import re
from tqdm import tqdm

appos = {
  "aren't" : "are not", "can't" : "cannot", "couldn't" : "could not", "didn't" : "did not",
  "doesn't" : "does not", "don't" : "do not", "hadn't" : "had not", "hasn't" : "has not",
  "haven't" : "have not", "he'd" : "he would", "he'll" : "he will", "he's" : "he is", "i'd" : "i would",
  "i'd" : "i had", "i'll" : "i will", "i'm" : "i am", "isn't" : "is not", "it's" : "it is", "it'll":"it will",
  "i've" : "I have", "let's" : "let us", "mightn't" : "might not", "mustn't" : "must not", "shan't" : "shall not",
  "she'd" : "she would", "she'll" : "she will", "she's" : "she is", "shouldn't" : "should not", "that's" : "that is",
  "there's" : "there is", "they'd" : "they would", "they'll" : "they will", "they're" : "they are", "they've" : "they have",
  "we'd" : "we would", "we're" : "we are", "weren't" : "were not", "we've" : "we have", "what'll" : "what will",
  "what're" : "what are", "what's" : "what is", "what've" : "what have", "where's" : "where is", "who'd" : "who would",
  "who'll" : "who will", "who're" : "who are", "who's" : "who is", "who've" : "who have", "won't" : "will not",
  "wouldn't" : "would not", "you'd" : "you would", "you'll" : "you will", "you're" : "you are", "you've" : "you have",
  "'re": " are", "wasn't": "was not", "we'll":" will", "didn't": "did not"
}

def clean_text(text):
  # Remove whitespaces and make strings lowercase
  text = text.strip().lower()
  words = text.split()
  # Nagation handling
  reformed = [appos[word] if word in appos else word for word in words]
  text = " ".join(reformed)
  pattern = '(@(\w+))'                # usermention (@username)
  pattern += '|(#(\w+))'              # hashtags (#somehashtag)
  pattern += '|([^\w\s])'             # emojis 😀
  pattern += '|(\\w+:\\/\\/\\S+)'     # urls (https://google.com)
  pattern += '|(\d+)'                 # numbers
  text = ' '.join(re.sub(pattern, ' ', text).split())
  return text

tqdm.pandas()
data = pd.read_csv('/content/drive/My Drive/SentimentAnalysis/rnn/data/data.csv', error_bad_lines=False)
data['CleanText'] = data['SentimentText'].progress_apply(lambda t: clean_text(t))
data.head()

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'
100%|██████████| 1578612/1578612 [00:40<00:00, 38950.34it/s]


,ItemID,Sentiment,SentimentSource,SentimentText,CleanText
0,1,0,Sentiment140,is so sad for my APL frie...,is so sad for my apl friend
1,2,0,Sentiment140,I missed the New Moon trail...,i missed the new moon trailer
2,3,1,Sentiment140,omg its already 7:30 :O,omg its already o
3,4,0,Sentiment140,.. Omgaga. Im sooo im gunna CRy. I'...,omgaga im sooo im gunna cry I have been at thi...
4,5,0,Sentiment140,i think mi bf is cheating on me!!! ...,i think mi bf is cheating on me t_t


In [3]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data['CleanText'], 
                                                    data['Sentiment'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=data['Sentiment'])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(1420750,) (157862,) (1420750,) (157862,)


In [0]:
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

In [0]:
def labelize_tweets_ug(tweets, label):
  result = []
  prefix = label
  for i, t in zip(tweets.index, tweets):
    result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
  return result

all_x = pd.concat([x_train, x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [15]:
cores = multiprocessing.cpu_count()
# Continuous Bag Of Words
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 1578612/1578612 [00:00<00:00, 2015741.49it/s]


In [16]:
%%time
for epoch in range(30):
  model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
  model_ug_cbow.alpha -= 0.002
  model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 1578612/1578612 [00:00<00:00, 2049045.99it/s]


CPU times: user 28min 4s, sys: 9.15 s, total: 28min 13s
Wall time: 15min 3s


In [17]:
# Skip Gram
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 1578612/1578612 [00:00<00:00, 2047530.95it/s]


In [18]:
%%time
for epoch in range(30):
  model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
  model_ug_sg.alpha -= 0.002
  model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 1578612/1578612 [00:00<00:00, 2060097.03it/s]


CPU times: user 49min 45s, sys: 9.23 s, total: 49min 54s
Wall time: 26min 5s


In [0]:
model_ug_cbow.save('/content/drive/My Drive/SentimentAnalysis/rnn/w2vmodels/w2v_model_ug_cbow.word2vec')
model_ug_sg.save('/content/drive/My Drive/SentimentAnalysis/rnn/w2vmodels/w2v_model_ug_sg.word2vec')

In [20]:
import numpy as np

embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 106091 word vectors.


In [21]:
from keras.preprocessing.text import Tokenizer

MAX_NB_WORDS = 80000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data['CleanText'])

Using TensorFlow backend.


In [0]:
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [0]:
# saving tokenizer
import pickle

with open('/content/drive/My Drive/SentimentAnalysis/rnn/tokenizers/tokenizer-rnn-cnn-w2v.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
from keras.preprocessing.sequence import pad_sequences

MAX_LENGTH = 35
padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LENGTH)
padded_test_sequences = pad_sequences(test_sequences, maxlen=MAX_LENGTH)
padded_train_sequences.shape

(1420750, 35)

In [0]:
embed_size = 200
# maximum number of words kept after tokenization based on their word frequency
MAX_NB_WORDS = 80000

num_words = MAX_NB_WORDS
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in tokenizer.word_index.items():
  if i >= num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [0]:
import numpy as np 
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, GRU, Bidirectional
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, Conv1D

def get_rnn_cnn_model():
  embedding_dim = 200
  inp = Input(shape=(MAX_LENGTH, ))
  x = Embedding(MAX_NB_WORDS, embedding_dim, weights=[embedding_matrix], input_length=MAX_LENGTH, trainable=True)(inp)
  x = SpatialDropout1D(0.3)(x)
  x = Bidirectional(GRU(100, return_sequences=True))(x)
  x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)

  model = Model(inputs=inp, outputs=outp)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [27]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

rnn_cnn_model = get_rnn_cnn_model()

filepath="/content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-cnn-w2v-model-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

batch_size = 256
epochs = 4

history = rnn_cnn_model.fit(x=padded_train_sequences, 
                    y=y_train, 
                    validation_data=(padded_test_sequences, y_test), 
                    batch_size=batch_size, 
                    callbacks=[checkpoint], 
                    epochs=epochs, 
                    verbose=1)

Train on 1420750 samples, validate on 157862 samples
Epoch 1/4
1420750/1420750 [==============================] - 1009s 710us/step - loss: 0.4039 - acc: 0.8153 - val_loss: 0.3707 - val_acc: 0.8362

Epoch 00001: val_acc improved from -inf to 0.83615, saving model to /content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-cnn-w2v-model-01-0.8362.hdf5
Epoch 2/4
1420750/1420750 [==============================] - 1002s 706us/step - loss: 0.3637 - acc: 0.8380 - val_loss: 0.3603 - val_acc: 0.8399

Epoch 00002: val_acc improved from 0.83615 to 0.83987, saving model to /content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-cnn-w2v-model-02-0.8399.hdf5
Epoch 3/4
1420750/1420750 [==============================] - 977s 688us/step - loss: 0.3459 - acc: 0.8477 - val_loss: 0.3604 - val_acc: 0.8419

Epoch 00003: val_acc improved from 0.83987 to 0.84191, saving model to /content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-cnn-w2v-model-03-0.8419.hdf5
Epoch 4/4
1420750/1420750 [==================